In [24]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Load data & drop unnamed columns
df = pd.read_csv('C:/airqual-project/AirQualityUCI.csv', sep=';', decimal=',', usecols=range(15))

# Convert date & time to datetime
df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format= '%d/%m/%Y %H.%M.%S', dayfirst=True)
df.drop(columns=['Date', 'Time', 'NMHC(GT)'], inplace=True) # Dropped NMHC(GT) as there's too many missing values & can't be used as reliable training data
df = df[['Datetime'] + [c for c in df if c not in ['Datetime']]]

# Handle missing values
df.replace(-200, np.nan, inplace=True)
df.dropna(subset=['Datetime'], inplace=True) # Drop rows with missing Datetime
df.sort_values('Datetime', inplace=True) 
df.set_index('Datetime', inplace=True)

# Drop rows with too many consecutive missing values
max_gap = 3
downtime_indices = [] 
consecutive_missing = []
for i in range(len(df)):
    if df.iloc[i].isnull().any():
        consecutive_missing.append(df.index[i])
    else:
        if len(consecutive_missing) > max_gap:
            downtime_indices.extend(consecutive_missing)
        consecutive_missing = [] 
if len(consecutive_missing) > max_gap:
    downtime_indices.extend(consecutive_missing)
df.drop(downtime_indices, inplace=True) 

# Interpolate missing values based on time
df.interpolate(method='time', inplace=True) 
print(df.isnull().sum()) # Confirm no missing values remain

# Standardize column names
df.rename(columns={'CO(GT)': 'co_gt', 
                   'PT08.S1(CO)': 'pt08_s1_co', 
                   'C6H6(GT)': 'c6h6_gt', 
                   'PT08.S2(NMHC)': 'pt08_s2_nmhc', 
                   'NOx(GT)': 'nox_gt', 
                   'PT08.S3(NOx)': 'pt08_s3_nox', 
                   'NO2(GT)': 'no2_gt', 
                   'PT08.S4(NO2)': 'pt08_s4_no2', 
                   'PT08.S5(O3)': 'pt08_s5_o3', 
                   'T': 't', 'RH': 'rh', 'AH': 'ah'}, inplace=True)

# Standardize data types
scaler = StandardScaler()
numeric_cols = df.select_dtypes(include=['float64']).columns
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

df.reset_index(inplace=True) 
df.info() # Confirm data is standardized & missing values are handled
df.head()
df.to_csv('C:/airqual-project/AirQualityUCI_cleaned.csv', index=False)

CO(GT)           0
PT08.S1(CO)      0
C6H6(GT)         0
PT08.S2(NMHC)    0
NOx(GT)          0
PT08.S3(NOx)     0
NO2(GT)          0
PT08.S4(NO2)     0
PT08.S5(O3)      0
T                0
RH               0
AH               0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7384 entries, 0 to 7383
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Datetime      7384 non-null   datetime64[ns]
 1   co_gt         7384 non-null   float64       
 2   pt08_s1_co    7384 non-null   float64       
 3   c6h6_gt       7384 non-null   float64       
 4   pt08_s2_nmhc  7384 non-null   float64       
 5   nox_gt        7384 non-null   float64       
 6   pt08_s3_nox   7384 non-null   float64       
 7   no2_gt        7384 non-null   float64       
 8   pt08_s4_no2   7384 non-null   float64       
 9   pt08_s5_o3    7384 non-null   float64       
 10  t             7384 non-null   float64       
 1